# Оценка использования основных фондов (ОФ) предприятия

## Исходные данные (вариант 10)

### Таблица 1

* Среднегодовая балансовая стоимость ОПФ (Фо): тыс. у.д.е.
* Средние фактические сроки службы ОПФ (Тф): лет         

In [ ]:
import pandas as pd

table1 = pd.DataFrame(data={
    'Фо': [19450, 1633, 945, 1084, 31200, 1740, 2232, 1212, 1016, 470],
    'Тф': [28, 16, 11, 12, 6, 5, 4, 6, 3, 8]
}, index=[
    'Здания',
    'Сооружения',
    'Передаточные устройства',
    'Силовые машины и оборудование',
    'Рабочие  машины и оборудование',
    'Измерительные и регулирующие приборы и устройства, лабораторн. обор-е',
    'Вычислительная техника',
    'Транспортные средства',
    'Инструмент',
    'Производственный и хозяйств. инвентарь и принадлежности'
])
table1.append(table1.select_dtypes(pd.np.number).sum().rename('ВСЕГО ОПФ'))

In [ ]:
table2 = pd.DataFrame(data={'Показатели': [17.4, 1994, 4, 12]}, index=[
    'Оптовая цена станка, тыс.руб.',
    'Год приобретения',
    'Транспортные расходы, % от оптовой цены',
    'Затраты на монтаж и отладку, % от оптовой цены'
])
# table2['Показатели']['Оптовая цена станка, тыс.руб.']
table2

In [ ]:
table3 = pd.DataFrame(data={'Показатели': [48, 12, 'август', 7, 'июль']}, index=[
    'Стоимость ОПФ на 1 января текущего года, тыс. у.д.е.',
    'Введено ОПФ в текущем году, тыс. у.д.е.',
    'Дата ввода',
    'Выбыло ОПФ в текущем году, тыс. у.д.е.',
    'Дата выбытия'
])
table3

In [ ]:
table4 = pd.DataFrame(data={
    'План': [132, 55, 24],
    'Отчет': [133.1, 52.2, 26]
}, index=[
    'Валовая продукция, тыс. у.д.е.',
    'Среднегодовая стоимость основных производственных фондов, тыс. у.д.е.',
    'Среднегодовая численность производственных рабочих, чел.',
])
table4

In [ ]:
table5 = pd.DataFrame(data={'Показатели': [25, 10, 1520, 7568]}, index=[
    'Количество станков в цехе',
    'Средняя часовая производительность одного станка, шт.',
    'Фактический выпуск изделий за смену, шт.',
    'Фактически отработано за месяц, станко-часов'
])
table5

## Определение структуры и состава основных фондов

### 1. Удельный вес $d$ отдельных групп ОФ в общей их стоимости

$$d = \frac{\overline{Ф}_{oi}}{\sum^n_{i=1} \overline{Ф}_{oi}}$$

где $\overline{Ф}_{oi}$ — среднегодовая стоимость $i$-й группы ОФ, $n$ — число групп.

In [ ]:
pd.DataFrame((table1['Фо'] / table1['Фо'].sum()).rename('Удельный вес $d$'))

### 2. Удельный вес активной части в общей стоимости ОФ

$$d_a = \frac{\sum^m_{i=1} \overline{Ф}_{oa}}{\sum^n_{i=1} \overline{Ф}_{oi}}$$

где $\overline{Ф}_{oi}$ — среднегодовая стоимость $i$-й группы активной части ОФ, $m$ — число групп в составе активной части ОФ (машины и оборудование, инструмент и др.)

In [ ]:
import numpy as np

np.round(table1['Фо'][2:].sum() / table1['Фо'].sum(), 4)

### 3. Удельный вес отдельных возрастных групп

$$d_j = \frac{\sum^k_{i=1} \overline{Ф}_{oi}}{\sum^n_{i=1} \overline{Ф}_{oi}}$$

где $k$ — число групп в составе $j$-й возрастной группы

In [ ]:
pd.DataFrame(data={'Удельный вес': [
    table1.query('Тф < 5')['Фо'].sum() / table1['Фо'].sum(),
    table1.query('Тф >= 5 & Тф < 10')['Фо'].sum() / table1['Фо'].sum(),
    table1.query('Тф >= 10 & Тф < 20')['Фо'].sum() / table1['Фо'].sum(),
    table1.query('Тф >= 20')['Фо'].sum()  / table1['Фо'].sum()
]}, index=[
    'до 5 лет', '5-10 лет', '10-20 лет', 'свыше 20 лет'
])

### Коэффициенты пересчета балансовой стоимости ОФ в восстановительную

In [ ]:
table1_cfs = pd.DataFrame(data={
    'нормы амортизац. отчислений, %': [2.00, 5.00, 5.00, 4.40, 7.00, 14.30, 12.00, 12.00, 25.00, 9.00],
    'коэффициент пересчета': [1.67, 1.71, 1.71, 1.10, 1.45, 1.15, 1.05, 1.35, 1.30, 1.05]
}, index=table1.index)
table1_cfs

### 4, 5, 6. Срок службы и износ

Примем $a$ — норма амортизационных отчислений, $T_{ф}$ — средний фактический срок службы.

* нормативный срок службы $$T_{н} = \frac{1}{a} \cdot 100$$
* коэффициент физического износа $$K_{фи} = \frac{T_{ф}}{T_{н}}$$
* износ $$И_i = \overline{Ф}_{oi} \cdot a \cdot T_{ф} \cdot \frac{1}{100}$$
* коэффициент износа $$K_{изн_i} = \frac{И_i}{\overline{Ф}_{oi}}$$

In [ ]:
standard_service_life = (table1_cfs['нормы амортизац. отчислений, %'] / 100).rename('$T_{н}$')
physical_wear_ratio = (standard_service_life / table1['Тф']).rename('$K_{фи}$')
wear = (table1['Фо'] * table1['Тф'] * table1_cfs['нормы амортизац. отчислений, %'] / 100).rename('$И_i$')
wear_factor = (wear / table1['Фо']).rename('$K_{изн_i}$')

pd.DataFrame(standard_service_life).join(physical_wear_ratio).join(wear).join(wear_factor)